<h1><b>Space X Falcon 9 First Stage Landing Prediction</b></h1>

## Step 3. EDA with SQL
In this step, we use SQL to gain more insights from our dataset.

## Objectives
<ul>
<li>Understand the Space X dataset</li>
<li>Load the dataset  into the corresponding table in a Postgre database</li>
<li>Execute SQL queries to gain more insights</li>
</ul>

<hr>

### Download the datasets

First, we should load the Space X dataset.

Here is the link to download and save the dataset (.CSV file): <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [1]:
#!pip install sqlalchemy
#!pip install ibm_db_sa
#!pip install ipython-sql
import csv, sqlite3
import pandas as pd

### Connect to the database

We load the SQL extension and establish a connection with the database.

In [2]:
%load_ext sql

In [3]:
con = sqlite3.connect("spacex.db")
cur = con.cursor()

In [4]:
%sql sqlite:///spacex.db

In [5]:
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               101 non-null    object 
 1   Time (UTC)         101 non-null    object 
 2   Booster_Version    101 non-null    object 
 3   Launch_Site        101 non-null    object 
 4   Payload            101 non-null    object 
 5   PAYLOAD_MASS__KG_  101 non-null    float64
 6   Orbit              101 non-null    object 
 7   Customer           101 non-null    object 
 8   Mission_Outcome    101 non-null    object 
 9   Landing_Outcome    101 non-null    object 
dtypes: float64(1), object(9)
memory usage: 78.2+ KB


In [6]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method="multi")

999

## Queries
##### 1. The names of unique launch sites in the space mission:

In [8]:
%%sql

SELECT DISTINCT Launch_Site 
FROM SPACEXTBL

 * sqlite:///spacex.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40
None



#####  2. The first 5 records where launch sites begin with the string 'CCA' :


In [10]:
%%sql

SELECT *
FROM SPACEXTBL
WHERE Launch_Site LIKE 'CCA%'
LIMIT 5

 * sqlite:///spacex.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
06/04/2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
12/08/2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22/05/2012,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
10/08/2012,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
03/01/2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt


##### 3. The total payload mass carried by boosters launched by NASA (CRS):


In [13]:
%%sql

SELECT SUM(Payload_Mass__KG_) AS TotalPayloadMass
FROM SPACEXTBL
WHERE Customer LIKE 'NASA (CRS)'

 * sqlite:///spacex.db
Done.


TotalPayloadMass
45596.0


##### 4. The Average payload mass carried by booster version F9 v1.1:


In [15]:
%%sql

SELECT AVG(Payload_Mass__KG_) AS Avg_PayloadMass
FROM SPACEXTBL
WHERE Booster_Version = 'F9 v1.1'

 * sqlite:///spacex.db
Done.


Avg_PayloadMass
2928.4


##### 5. The date of when the first succesful landing outcome in ground pad was acheived:

In [17]:
%%sql

SELECT MIN(Date) AS FirstSuccessfullLandingDate
FROM SPACEXTBL
WHERE Landing_Outcome LIKE 'Success (ground pad)'

 * sqlite:///spacex.db
Done.


FirstSuccessfullLandingDate
01/08/2018


##### 6. The names of the boosters that have succeded in drone ship and have a payload mass greater than 4000 but less than 6000:


In [18]:
%%sql

SELECT Booster_Version
FROM SPACEXTBL
WHERE Landing_Outcome = 'Success (drone ship)'
    AND Payload_Mass__KG_ > 4000 
    AND Payload_Mass__KG_ < 6000

 * sqlite:///spacex.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


##### 7. The total number of successful and failed mission outcomes:


In [42]:
%%sql

SELECT 
    COUNT(CASE WHEN Mission_Outcome LIKE 'Success%' THEN 1 END) AS SuccessOutcome,
    COUNT(CASE WHEN Mission_Outcome LIKE 'Failure%' THEN 1 END) AS FailureOutcome
FROM SPACEXTBL;


 * sqlite:///spacex.db
Done.


SuccessOutcome,FailureOutcome
100,1


##### 8. The names of the booster versions which have carried the maximum payload mass:


In [47]:
%%sql

SELECT Booster_Version
        FROM SPACEXTBL
        WHERE Payload_Mass__KG_ = (
                                SELECT MAX(Payload_Mass__KG_)
                                FROM SPACEXTBL
                                )
        ORDER BY Booster_Version

 * sqlite:///spacex.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1048.5
F9 B5 B1049.4
F9 B5 B1049.5
F9 B5 B1049.7
F9 B5 B1051.3
F9 B5 B1051.4
F9 B5 B1051.6
F9 B5 B1056.4
F9 B5 B1058.3


##### 9. The records of 2015 with their months, failure landing outcomes in drone ship, booster versions and launch sites.

In [68]:
%%sql

SELECT Booster_Version, Launch_Site, Landing_Outcome, substr(Date, 4, 2) AS Month
FROM SPACEXTBL
WHERE Landing_Outcome LIKE 'Failure (drone ship)'
AND substr(Date,7,4)='2015'

 * sqlite:///spacex.db
Done.


Booster_Version,Launch_Site,Landing_Outcome,Month
F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship),10
F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship),04


##### 10. The count of  successful landing outcomes between the date 04-06-2010 and 20-03-2017 in descending order:

In [73]:
%%sql

SELECT Landing_Outcome, COUNT(Landing_Outcome)
FROM SPACEXTBL
WHERE DATE BETWEEN '04-06-2010' AND '20-03-2017'
AND Landing_Outcome LIKE 'Success%'
GROUP BY Landing_Outcome
ORDER BY COUNT(Landing_Outcome) DESC

 * sqlite:///spacex.db
Done.


Landing_Outcome,COUNT(Landing_Outcome)
Success,20
Success (drone ship),8
Success (ground pad),7
